<a href="https://colab.research.google.com/github/patelmahir649/BCI2a-EEG-NN-Pipeline/blob/main/Copy_of_rbfModel_redone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import os
import scipy
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset as TData
from sklearn.model_selection import train_test_split as tts
import pickle
import zipfile
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split as tts

In [7]:
!pip install optuna

import optuna
from optuna.trial import TrialState

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 21.7 MB/s eta 0:00:00


In [2]:
!unzip "LHNT_Individual_Data.zip"

Archive:  LHNT_Individual_Data.zip
  inflating: LHNT EEG/Alan_Fletcher_Session4.zip  
  inflating: LHNT EEG/Ademola_Adetosoye_Session1.0.zip  
  inflating: LHNT EEG/Alex_Xie_Session1.zip  
  inflating: LHNT EEG/Alex_Xie_Session2.zip  
  inflating: LHNT EEG/Alan_Fletcher_Session2.zip  
  inflating: LHNT EEG/Alex_Xie_Session3.zip  
  inflating: LHNT EEG/Alan_Fletcher_Session1.zip  
  inflating: LHNT EEG/Alan_Fletcher_Session3.zip  
  inflating: LHNT EEG/Alex_Xie_Session4.zip  
  inflating: LHNT EEG/Jane_Doe_Session407.0.zip  
  inflating: LHNT EEG/jimmy_neutron_Session3.0.zip  
  inflating: LHNT EEG/jimmy_neutron_Session4.0.zip  
  inflating: LHNT EEG/Maddox_Fletcher_Session1.0.zip  
  inflating: LHNT EEG/morgan_dye_Session1.zip  
  inflating: LHNT EEG/morgan_dye_Session2.zip  
  inflating: LHNT EEG/morgan_dye_Session3.zip  
  inflating: LHNT EEG/nandini_senthilkumar_Session1.zip  
  inflating: LHNT EEG/nandini_senthilkumar_Session2.zip  
  inflating: LHNT EEG/nandini_senthilkumar_Sessio

In [8]:
def getAllPickles(directory="LHNT EEG"):
    # List all subfolders and collect paths to all .pkl files.
    folders = [drctry for drctry in os.listdir(directory) if os.path.isdir(os.path.join(directory, drctry))]
    files = []
    for folder in folders:
        folder_files = os.listdir(os.path.join(directory, folder))
        for file in folder_files:
            if ".pkl" in file:
                files.append(os.path.join(directory, folder, file))
    return files

def npFromPickle(pickle_files):
    # Load each pickle file and assign a label:
    # 1 if filename contains "right", else 0.
    np_data = []
    labels = []  # 0 is left, 1 is right
    for file in pickle_files:
        with open(file, "rb") as f:
            data1 = pickle.load(f)
            np_data.append(data1[0])
        if 'right' in file.split('/')[-1]:
            labels.append(1)
        else:
            labels.append(0)
    return np_data, labels

np_data, labels = npFromPickle(getAllPickles())
print("Total samples:", len(np_data), "Total labels:", len(labels))

Total samples: 0 Total labels: 0


In [9]:
def bandpass_filter(signal, crit_freq=[1, 40], sampling_freq=125, plot=False, channel=0):
    # Applies a Butterworth bandpass filter.
    order = 4
    b, a = scipy.signal.butter(order, crit_freq, btype='bandpass', fs=sampling_freq)
    processed_signal = scipy.signal.filtfilt(b, a, signal, 1)
    if plot:
        plt.figure()
        plt.xlabel('Time')
        plt.ylabel(f'Normalized amplitude of channel {channel}')
        plt.title(f'{crit_freq[0]}-{crit_freq[1]}Hz bandpass filter')
        signal_min = np.full((signal.shape[1], signal.shape[0]), np.min(signal, 1)).transpose()
        signal_max = np.full((signal.shape[1], signal.shape[0]), np.max(signal, 1)).transpose()
        normed_signal = (signal - signal_min) / (signal_max - signal_min)
        filtered_min = np.full((processed_signal.shape[1], processed_signal.shape[0]), np.min(processed_signal, 1)).transpose()
        filtered_max = np.full((processed_signal.shape[1], processed_signal.shape[0]), np.max(processed_signal, 1)).transpose()
        normed_filt = (processed_signal - filtered_min) / (filtered_max - filtered_min)
        plt.plot(np.arange(normed_signal[channel].size), normed_signal[channel], label='Input')
        plt.plot(np.arange(normed_filt[channel].size), normed_filt[channel], label='Transformed')
        plt.legend()
    return processed_signal

def segmentation(signal, sampling_freq=125, window_size=1, window_shift=0.016):
    # Segments the signal into overlapping windows.
    w_size = int(sampling_freq * window_size)
    w_shift = int(sampling_freq * window_shift)
    segments = []
    i = 0
    while i + w_size <= signal.shape[1]:
        segments.append(signal[:, i: i + w_size])
        i += w_shift
    return segments

def channel_rearrangment(sig, channel_order):
    # Rearranges channels according to the provided order.
    # The channel_order is given in 1-indexed format.
    channel_order = [channel - 1 for channel in channel_order]
    reindexed = np.zeros_like(sig)
    for i, ind in enumerate(channel_order):
        reindexed[i] = sig[ind]
    return reindexed

# Define desired channel order.
ordered_channels = [1, 9, 11, 3, 2, 12, 10, 4, 13, 5, 15, 7, 14, 16, 6, 8]

In [10]:
class FixedGammaRBFLayer(nn.Module):
    def __init__(self, in_features, out_features, gamma=0.1):
        super(FixedGammaRBFLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.gamma = gamma
        self.centers = nn.Parameter(torch.Tensor(out_features, in_features))
        self.weights = nn.Parameter(torch.ones(out_features))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.centers)
        nn.init.constant_(self.weights, 1.0)

    def forward(self, x):
        # x: (batch, in_features)
        x_expanded = x.unsqueeze(1)                 # (batch, 1, in_features)
        centers_expanded = self.centers.unsqueeze(0)  # (1, out_features, in_features)
        diff = x_expanded - centers_expanded
        dist_sq = torch.sum(diff ** 2, dim=2)         # (batch, out_features)
        rbf_out = torch.exp(-self.gamma * dist_sq)    # RBF activation.
        rbf_out = rbf_out * self.weights              # Scale by trainable weights.
        return rbf_out

class RBFNetwork(nn.Module):
    def __init__(self, input_dim, num_rbf_units, num_classes,
                 gamma=0.1, hidden_dim=512, dropout_prob=0.5):
        super(RBFNetwork, self).__init__()
        # RBF layer: transforms flattened input into RBF activations.
        self.rbf = FixedGammaRBFLayer(input_dim, num_rbf_units, gamma=gamma)
        self.dropout = nn.Dropout(dropout_prob)
        # Classifier: two-layer FC network with ReLU and dropout.
        self.fc = nn.Sequential(
            nn.Linear(num_rbf_units, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_prob),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        # Flatten input: (batch, channels, sequence_length) -> (batch, input_dim)
        x = x.view(x.size(0), -1)
        x_rbf = self.rbf(x)
        out = self.fc(x_rbf)
        return out

def initialize_rbf_centers(model, data_loader, num_samples=1000):
    # Use KMeans clustering on a subset of training data to initialize RBF centers.
    model.eval()
    all_features = []
    total_samples = 0
    for features, _ in data_loader:
        features = features.to(device)
        features = features.view(features.size(0), -1)
        with torch.no_grad():
            all_features.append(features.cpu())
        total_samples += features.size(0)
        if total_samples >= num_samples:
            break
    all_features = torch.cat(all_features, dim=0)
    kmeans = KMeans(n_clusters=model.rbf.out_features, random_state=0)
    kmeans.fit(all_features.numpy())
    centers = torch.tensor(kmeans.cluster_centers_, dtype=torch.float32, device=device)
    model.rbf.centers.data.copy_(centers)
    print("RBF centers initialized using KMeans on input features.")

def train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs=20, patience=5):
    best_valid_accuracy = 0.0
    no_improve_epochs = 0

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        for features, labels in train_loader:
            features = features.to(device)
            labels = labels.argmax(dim=1).to(device)
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        avg_train_loss = train_loss / len(train_loader)

        # Validation phase
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for features, labels in valid_loader:
                features = features.to(device)
                labels = labels.argmax(dim=1).to(device)
                outputs = model(features)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        valid_accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Valid Accuracy: {valid_accuracy:.2f}%")

        # Early stopping check
        if valid_accuracy > best_valid_accuracy:
            best_valid_accuracy = valid_accuracy
            no_improve_epochs = 0
        else:
            no_improve_epochs += 1
            if no_improve_epochs >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    return best_valid_accuracy

def evaluate_model(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for features, labels in data_loader:
            features = features.to(device)
            labels = labels.argmax(dim=1).to(device)
            outputs = model(features)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy:.2f}%")
    return accuracy

def objective(trial):
    # Hyperparameters to optimize
    num_rbf_units = trial.suggest_int("num_rbf_units", 64, 512, step=64)
    gamma = trial.suggest_float("gamma", 0.001, 1.0, log=True)
    hidden_dim = trial.suggest_int("hidden_dim", 128, 1024, step=128)
    dropout_prob = trial.suggest_float("dropout_prob", 0.1, 0.7)
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])

    # Create DataLoaders with the suggested batch size
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_ds, batch_size=batch_size, shuffle=False)

    # Initialize model with the suggested hyperparameters
    model = RBFNetwork(
        input_dim=input_dim,
        num_rbf_units=num_rbf_units,
        num_classes=num_classes,
        gamma=gamma,
        hidden_dim=hidden_dim,
        dropout_prob=dropout_prob
    ).to(device)

    # Initialize RBF centers using KMeans on training data
    initialize_rbf_centers(model, train_loader, num_samples=500)

    # Setup optimizer with suggested learning rate and weight decay
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()

    # Train and validate the model
    valid_accuracy = train_model(
        model=model,
        train_loader=train_loader,
        valid_loader=valid_loader,
        criterion=criterion,
        optimizer=optimizer,
        num_epochs=20,
        patience=5
    )

    return valid_accuracy

In [19]:
import os
import zipfile
import pickle
import numpy as np
from sklearn.model_selection import train_test_split as tts
import shutil

# Create a directory for extracting the zip files
extract_dir = "extracted_eeg_data"
if os.path.exists(extract_dir):
    shutil.rmtree(extract_dir)  # Remove if exists to start fresh
os.makedirs(extract_dir)

# Extract all zip files
print("Extracting ZIP files...")
zip_count = 0
for zip_file in os.listdir('LHNT EEG'):
    if zip_file.endswith('.zip'):
        zip_count += 1
        zip_path = os.path.join('LHNT EEG', zip_file)
        # Create a specific directory for each zip file to avoid file collisions
        session_extract_dir = os.path.join(extract_dir, os.path.splitext(zip_file)[0])
        os.makedirs(session_extract_dir, exist_ok=True)

        try:
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(session_extract_dir)
            print(f"Extracted: {zip_file}")
        except Exception as e:
            print(f"Error extracting {zip_file}: {e}")

print(f"\nExtracted {zip_count} ZIP files")

# Now look for pickle files in the extracted directories
def getAllPickles(directory):
    """Find all pickle files in directory and subdirectories."""
    all_files = []

    # Walk through all subdirectories
    for root, dirs, files in os.walk(directory):
        if "__MACOSX" in root:  # Skip macOS system directories
            continue

        for file in files:
            if file.endswith(".pkl") and not file.startswith("._"):
                all_files.append(os.path.join(root, file))

    print(f"Found {len(all_files)} pickle files")
    return all_files

def npFromPickle(pickle_files, debug=True):
    """Load data from pickle files and return data, labels and shapes."""
    np_data = []
    labels = [] # 0 is left, 1 is right
    shapes = []

    for file in pickle_files:
        if "__MACOSX" in file:  # Skip macOS system files
            continue

        try:
            with open(file, "rb") as f:
                data1 = pickle.load(f)
                np_data.append(data1[0])
                shapes.append(data1[0].shape)

            # Check for 'right' or 'left' in filename
            filename = os.path.basename(file).lower()
            if 'right' in filename:
                labels.append(1)
            elif 'left' in filename:
                labels.append(0)
            else:
                print(f"Warning: Cannot determine left/right from filename: {file}")
                # Skip this file or use another method to determine label
                np_data.pop()  # Remove the corresponding data
                shapes.pop()   # Remove the corresponding shape
                continue

        except Exception as e:
            print(f"Error loading {file}: {e}")

    if debug and shapes:
        # Find common shapes
        from collections import Counter
        shape_counts = Counter(shapes)
        print(f"Found {len(shape_counts)} different shapes in the data")
        for shape, count in shape_counts.most_common(5):
            print(f"Shape {shape}: {count} samples")

    return np_data, labels, shapes

# Get pickle files from extracted directories
pickle_files = getAllPickles(extract_dir)

if not pickle_files:
    print("No pickle files found in extracted directories.")

    # Let's look for any files to understand the structure
    print("\nExamining extracted directory structure:")
    for root, dirs, files in os.walk(extract_dir):
        if files:
            print(f"Directory: {root}")
            # Show first 5 files in each directory with files
            for file in files[:5]:
                print(f"  - {file}")
            if len(files) > 5:
                print(f"  - ... and {len(files)-5} more files")

    print("\nFile extensions in extracted data:")
    extensions = {}
    for root, dirs, files in os.walk(extract_dir):
        for file in files:
            ext = os.path.splitext(file)[1].lower()
            extensions[ext] = extensions.get(ext, 0) + 1

    for ext, count in extensions.items():
        print(f"{ext}: {count} files")

else:
    # Process and load the data
    np_data, labels, shapes = npFromPickle(pickle_files)
    print(f"Loaded {len(np_data)} samples with {len(labels)} labels")

    if not shapes or len(shapes) == 0:
        print("No valid data shapes found. Check your pickle files.")
    else:
        # Now let's handle the different shapes
        # Option 1: Keep only samples with the most common shape
        from collections import Counter
        shape_counts = Counter(shapes)
        most_common_shape = shape_counts.most_common(1)[0][0]
        print(f"Most common shape: {most_common_shape}")

        # Filter data to only include samples with the most common shape
        filtered_indices = [i for i, shape in enumerate(shapes) if shape == most_common_shape]
        filtered_data = [np_data[i] for i in filtered_indices]
        filtered_labels = [labels[i] for i in filtered_indices]

        print(f"After filtering: {len(filtered_data)} samples with consistent shape")

        # Convert filtered data to numpy arrays
        np_data_array = np.array(filtered_data)
        np_labels = np.array(filtered_labels)

        print(f"Data array shape: {np_data_array.shape}")

        # Now do the train-test split
        train_x, test_x, train_y, test_y = tts(np_data_array, np_labels, test_size=0.25, random_state=42)
        val_x, test_x = test_x[:len(test_x)//2], test_x[len(test_x)//2:]
        val_y, test_y = test_y[:len(test_y)//2], test_y[len(test_y)//2:]

        print(f"\nSplit data:")
        print(f"Train: {train_x.shape} with {sum(train_y)} right and {len(train_y) - sum(train_y)} left samples")
        print(f"Validation: {val_x.shape} with {sum(val_y)} right and {len(val_y) - sum(val_y)} left samples")
        print(f"Test: {test_x.shape} with {sum(test_y)} right and {len(test_y) - sum(test_y)} left samples")


Extracting ZIP files...
Extracted: Alex_Xie_Session2.zip
Extracted: Samarth_Rao_Session5.0.zip
Extracted: jimmy_neutron_Session4.0.zip
Extracted: morgan_dye_Session1.zip
Extracted: nandini_senthilkumar_Session4.zip
Extracted: morgan_dye_Session2.zip
Extracted: morgan_dye_Session3.zip
Extracted: Samarth_Rao_Session3.0.zip
Extracted: Samarth_Rao_Session1.0.zip
Extracted: Nandini_Senthilkumar_Session6.zip
Extracted: Jane_Doe_Session407.0.zip
Extracted: Alan_Fletcher_Session2.zip
Extracted: Samarth_Rao_Session2.0.zip
Extracted: Ademola_Adetosoye_Session1.0.zip
Extracted: jimmy_neutron_Session3.0.zip
Extracted: Alex_Xie_Session1.zip
Extracted: nandini_senthilkumar_Session1.zip
Extracted: Alan_Fletcher_Session4.zip
Extracted: Alan_Fletcher_Session1.zip
Extracted: Samarth_Rao_Session4.0.zip
Extracted: Alex_Xie_Session4.zip
Extracted: nandini_senthilkumar_Session2.zip
Extracted: Alex_Xie_Session3.zip
Extracted: Maddox_Fletcher_Session1.0.zip
Extracted: Nandini_Senthilkumar_Session5.zip
Extract

In [20]:
    # Preprocess and segment data
    train_eeg = []
    train_labels = []
    valid_eeg = []
    valid_labels = []
    test_eeg = []
    test_labels = []

    for sig, label in zip(train_x, train_y):
        if sig.shape[1] == 0:
            continue
        reindexed_signal = channel_rearrangment(sig, ordered_channels)
        filtered_sig = bandpass_filter(reindexed_signal, [5, 40], 125)
        normed_sig = (filtered_sig - np.mean(filtered_sig, 1, keepdims=True)) / np.std(filtered_sig, 1, keepdims=True)
        if np.isnan(normed_sig).any():
            continue
        signals = segmentation(normed_sig, 125, window_size=1.5, window_shift=0.0175)
        train_eeg.extend(signals)
        train_labels.extend([label] * len(signals))

    for sig, label in zip(val_x, val_y):
        if sig.shape[1] == 0:
            continue
        reindexed_signal = channel_rearrangment(sig, ordered_channels)
        filtered_sig = bandpass_filter(reindexed_signal, [5, 40], 125)
        normed_sig = (filtered_sig - np.mean(filtered_sig, 1, keepdims=True)) / np.std(filtered_sig, 1, keepdims=True)
        if np.isnan(normed_sig).any():
            continue
        signals = segmentation(normed_sig, 125, window_size=1.5, window_shift=0.0175)
        valid_eeg.extend(signals)
        valid_labels.extend([label] * len(signals))

    for sig, label in zip(test_x, test_y):
        if sig.shape[1] == 0:
            continue
        reindexed_signal = channel_rearrangment(sig, ordered_channels)
        filtered_sig = bandpass_filter(reindexed_signal, [5, 40], 125)
        normed_sig = (filtered_sig - np.mean(filtered_sig, 1, keepdims=True)) / np.std(filtered_sig, 1, keepdims=True)
        if np.isnan(normed_sig).any():
            continue
        signals = segmentation(normed_sig, 125, window_size=1.5, window_shift=0.0175)
        test_eeg.extend(signals)
        test_labels.extend([label] * len(signals))

    # Remove some channels to reduce dimensionality
    columns_to_remove = [1, 2, 7, 8]
    train_eeg = [np.delete(arr, columns_to_remove, axis=0) for arr in train_eeg]
    valid_eeg = [np.delete(arr, columns_to_remove, axis=0) for arr in valid_eeg]
    test_eeg = [np.delete(arr, columns_to_remove, axis=0) for arr in test_eeg]

    # Convert data to PyTorch tensors
    train_eeg_tensor = torch.zeros((len(train_eeg), train_eeg[0].shape[0], train_eeg[0].shape[1]))
    valid_eeg_tensor = torch.zeros((len(valid_eeg), valid_eeg[0].shape[0], valid_eeg[0].shape[1]))
    test_eeg_tensor = torch.zeros((len(test_eeg), test_eeg[0].shape[0], test_eeg[0].shape[1]))

    for i in range(len(train_eeg)):
        train_eeg_tensor[i] = torch.from_numpy(train_eeg[i].copy())
    for i in range(len(valid_eeg)):
        valid_eeg_tensor[i] = torch.from_numpy(valid_eeg[i].copy())
    for i in range(len(test_eeg)):
        test_eeg_tensor[i] = torch.from_numpy(test_eeg[i].copy())

    # Create one-hot encoded labels
    num_classes = 2
    train_label_tensor = torch.zeros(len(train_labels), num_classes)
    valid_label_tensor = torch.zeros(len(valid_labels), num_classes)
    test_label_tensor = torch.zeros(len(test_labels), num_classes)

    for i, val in enumerate(train_labels):
        train_label_tensor[i][val] = 1
    for i, val in enumerate(valid_labels):
        valid_label_tensor[i][val] = 1
    for i, val in enumerate(test_labels):
        test_label_tensor[i][val] = 1

    # Create TensorDatasets
    train_ds = TData(train_eeg_tensor, train_label_tensor)
    valid_ds = TData(valid_eeg_tensor, valid_label_tensor)
    test_ds = TData(test_eeg_tensor, test_label_tensor)

    # Determine input dimensions and number of classes
    sample_feature, sample_label = train_ds[0]
    input_dim = sample_feature.numel()  # Flattened input dimension
    num_classes = sample_label.shape[0]  # Number of classes

    # Use GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Set up Optuna study for hyperparameter optimization
    study = optuna.create_study(direction="maximize",
                               study_name="rbf_network_optimization",
                               pruner=optuna.pruners.MedianPruner())
    study.optimize(objective, n_trials=50)  # Number of trials for hyperparameter search

    # Print optimization results
    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print(f"  Value: {trial.value}")
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

    # Train final model with best hyperparameters
    print("\nTraining final model with best hyperparameters...")

    # Create DataLoaders for the final model
    final_batch_size = trial.params["batch_size"]
    train_loader = DataLoader(train_ds, batch_size=final_batch_size, shuffle=True)
    valid_loader = DataLoader(valid_ds, batch_size=final_batch_size, shuffle=False)
    test_loader = DataLoader(test_ds, batch_size=final_batch_size, shuffle=False)

    # Initialize the best model
    best_model = RBFNetwork(
        input_dim=input_dim,
        num_rbf_units=trial.params["num_rbf_units"],
        num_classes=num_classes,
        gamma=trial.params["gamma"],
        hidden_dim=trial.params["hidden_dim"],
        dropout_prob=trial.params["dropout_prob"]
    ).to(device)

    # Initialize RBF centers
    initialize_rbf_centers(best_model, train_loader, num_samples=500)

    # Set up optimizer and criterion
    best_optimizer = optim.Adam(
        best_model.parameters(),
        lr=trial.params["lr"],
        weight_decay=trial.params["weight_decay"]
    )
    criterion = nn.CrossEntropyLoss()

    # Train the final model
    train_model(
        model=best_model,
        train_loader=train_loader,
        valid_loader=valid_loader,
        criterion=criterion,
        optimizer=best_optimizer,
        num_epochs=30,  # Train for longer for the final model
        patience=10
    )

    # Evaluate on validation set
    print("\nValidation Set Evaluation:")
    valid_accuracy = evaluate_model(best_model, valid_loader)

    # Evaluate on test set
    print("\nTest Set Evaluation:")
    test_accuracy = evaluate_model(best_model, test_loader)

    # Save the best model
    torch.save(best_model.state_dict(), "best_rbf_network.pth")
    print("Best model saved to best_rbf_network.pth")

    # Save the study
    import joblib
    joblib.dump(study, "optuna_study.pkl")
    print("Optuna study saved to optuna_study.pkl")

[I 2025-04-26 23:02:11,329] A new study created in memory with name: rbf_network_optimization


Using device: cuda
RBF centers initialized using KMeans on input features.
Epoch 1/20 | Train Loss: 0.6930 | Valid Accuracy: 42.86%
Epoch 2/20 | Train Loss: 0.6929 | Valid Accuracy: 42.86%
Epoch 3/20 | Train Loss: 0.6928 | Valid Accuracy: 42.86%
Epoch 4/20 | Train Loss: 0.6929 | Valid Accuracy: 42.86%
Epoch 5/20 | Train Loss: 0.6927 | Valid Accuracy: 42.86%


[I 2025-04-26 23:03:02,679] Trial 0 finished with value: 42.857142857142854 and parameters: {'num_rbf_units': 128, 'gamma': 0.07753921665396608, 'hidden_dim': 1024, 'dropout_prob': 0.6899455042489528, 'lr': 0.0005228299729168187, 'weight_decay': 8.026967328195932e-05, 'batch_size': 128}. Best is trial 0 with value: 42.857142857142854.


Epoch 6/20 | Train Loss: 0.6928 | Valid Accuracy: 42.86%
Early stopping at epoch 6
RBF centers initialized using KMeans on input features.
Epoch 1/20 | Train Loss: 0.6943 | Valid Accuracy: 42.86%
Epoch 2/20 | Train Loss: 0.6931 | Valid Accuracy: 42.86%
Epoch 3/20 | Train Loss: 0.6933 | Valid Accuracy: 42.86%
Epoch 4/20 | Train Loss: 0.6934 | Valid Accuracy: 42.86%
Epoch 5/20 | Train Loss: 0.6934 | Valid Accuracy: 42.86%


[I 2025-04-26 23:03:34,436] Trial 1 finished with value: 42.857142857142854 and parameters: {'num_rbf_units': 64, 'gamma': 0.005576329386090816, 'hidden_dim': 1024, 'dropout_prob': 0.6980811904720657, 'lr': 0.008916157870479833, 'weight_decay': 0.00028541510866998183, 'batch_size': 64}. Best is trial 0 with value: 42.857142857142854.


Epoch 6/20 | Train Loss: 0.6935 | Valid Accuracy: 42.86%
Early stopping at epoch 6
RBF centers initialized using KMeans on input features.
Epoch 1/20 | Train Loss: 0.6925 | Valid Accuracy: 42.86%
Epoch 2/20 | Train Loss: 0.6926 | Valid Accuracy: 42.86%
Epoch 3/20 | Train Loss: 0.6925 | Valid Accuracy: 42.86%
Epoch 4/20 | Train Loss: 0.6925 | Valid Accuracy: 42.86%
Epoch 5/20 | Train Loss: 0.6926 | Valid Accuracy: 42.86%


[I 2025-04-26 23:05:42,826] Trial 2 finished with value: 42.857142857142854 and parameters: {'num_rbf_units': 384, 'gamma': 0.07036171090090922, 'hidden_dim': 768, 'dropout_prob': 0.26317007976214823, 'lr': 0.00010253247109351304, 'weight_decay': 0.0009481378735028546, 'batch_size': 64}. Best is trial 0 with value: 42.857142857142854.


Epoch 6/20 | Train Loss: 0.6926 | Valid Accuracy: 42.86%
Early stopping at epoch 6
RBF centers initialized using KMeans on input features.
Epoch 1/20 | Train Loss: 0.6930 | Valid Accuracy: 42.86%
Epoch 2/20 | Train Loss: 0.6928 | Valid Accuracy: 42.86%
Epoch 3/20 | Train Loss: 0.6928 | Valid Accuracy: 42.86%
Epoch 4/20 | Train Loss: 0.6929 | Valid Accuracy: 42.86%
Epoch 5/20 | Train Loss: 0.6928 | Valid Accuracy: 42.86%


[W 2025-04-26 23:07:34,971] Trial 3 failed with parameters: {'num_rbf_units': 320, 'gamma': 0.01420204644482993, 'hidden_dim': 896, 'dropout_prob': 0.3982240551447057, 'lr': 0.0035244201654876764, 'weight_decay': 2.308145741787735e-05, 'batch_size': 32} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "<ipython-input-10-377153e2782d>", line 162, in objective
    valid_accuracy = train_model(
                     ^^^^^^^^^^^^
  File "<ipython-input-10-377153e2782d>", line 83, in train_model
    train_loss += loss.item()
                  ^^^^^^^^^^^
KeyboardInterrupt
[W 2025-04-26 23:07:34,973] Trial 3 failed with value None.


KeyboardInterrupt: 